In [2]:
with open('BofM.txt') as f:
    text = f.readlines()

bofm = []
extra = ''
flag = False
count = 0
verse = ''
for x in text:
    if x[0] == '\n' and flag == True:
        bofm.append({
                'verse':verse,
                'ref':ref
                })
        verse = ''
        flag = False
    elif x[0] == '\n':
        flag = False
        verse = ''
    elif x[0] == ' ' or flag == True:
        flag = True
        y = x.replace('\n',' ')
        if y[2].isdigit() == True:
            verse += y[3:]
        elif y[1].isdigit() == True:
            verse += y[2:]
        else:
            verse += y
        if y[0] == ' ':
            ref = text[count - 1].replace('\n','')
    else:
        extra += x
    count += 1

import pandas as pd
test = pd.DataFrame(bofm)

import re
def remove_punct(cell):
    return ' '.join(re.findall(r"[\w']+", cell))

test.verse = test.verse.map(remove_punct)

bofm = ''
for x in test.verse:
    bofm += x + ' '

wordcount = pd.Series(bofm.split(' ')).str.lower()

from nltk.corpus import stopwords
s=set(stopwords.words('english'))

real_words = wordcount[~wordcount.isin(s)].copy()

real_words = real_words.reset_index(drop=True)

In [2]:
#the raw text file of the bofm
#text

#dataframe of references and verses of the entire bofm
#test

#string of the entire text of the bofm without punctuation
#bofm

#series of all words in the bofm
#wordcount

#list of the default stop words from the NLTK corpus
#s

#list of words that are left after removing punctuation and common words
#real_words

In [3]:
def return_book(cell):
    if cell.find('Words') > -1:
        return 'Words of Mormon'
    else:
        start1 = re.search('[a-z]',cell).start(0)
        endofbook = cell.find(' ',start1)
        startofbook = 0
        return cell[startofbook:endofbook]
def return_chapter(cell):
    if cell.find('Words') > -1:
        start_list = [m.start() for m in re.finditer('\s',cell)]
        return cell[start_list[-1]:].split(':')[0]
    else:
        start1 = re.search('[a-z]',cell).start(0)
        endofbook = cell.find(' ',start1)
        return cell[endofbook:].split(':')[0]
def return_verse(cell):
    start1 = re.search('[a-z]',cell).start(0)
    endofbook = cell.find(' ',start1)
    return cell[endofbook:].split(':')[1]
def return_words(cell):
    word_list = cell.split(' ')
    return len(word_list)

In [10]:
# start_list = [m.start() for m in re.finditer('\s',test.loc[0,'ref'])]

In [4]:
test['book'] = test.ref.map(return_book)

In [5]:
test['chapter'] = test.ref.map(return_chapter)

In [6]:
test['verseNum'] = test.ref.map(return_verse)

In [7]:
test['words'] = test.verse.map(return_words)

In [8]:
test['characters'] = test.verse.map(lambda x: len(x.replace(' ','')))

In [9]:
test.chapter = test.chapter.astype(int)
test.verseNum = test.verseNum.astype(int)
test.words = test.words.astype(int)
test.characters = test.characters.astype(int)

In [9]:
summarydf = pd.DataFrame(0,index=test.book.unique(),columns=['chapters','verses','words','characters',
                                                             'V/C','W/V','Char/W'])
for book in test.book.unique():
    temp = test[test.book == book].copy()
    chapters = len(temp.chapter.unique())
    verses = temp.book.count()
    words = temp.words.sum()
    characters = temp.characters.sum()
    avgCharactersPerWord = round(characters / float(words),2)
    avgWordsPerVerse = int(words / float(verses))
    avgVersePerChapter = int(verses / float(chapters))
    summarydf.loc[book,'chapters'] = chapters
    summarydf.loc[book,'verses'] = verses
    summarydf.loc[book,'words'] = words
    summarydf.loc[book,'characters'] = characters
    summarydf.loc[book,'W/V'] = avgWordsPerVerse
    summarydf.loc[book,'V/C'] = avgVersePerChapter
    summarydf.loc[book,'Char/W'] = avgCharactersPerWord

In [10]:
summarydf2 = pd.DataFrame(0,index=['Total','Average'],columns=summarydf.columns)
                          
for x in summarydf.columns:
    summarydf2.loc['Total',x] = int(summarydf[x].sum())

for x in summarydf.columns:
    summarydf2.loc['Average',x] = int(summarydf[x].mean())

In [11]:
summarydf

,chapters,verses,words,characters,V/C,W/V,Char/W
1 Nephi,22,617,25031,101729,28,40,4.06
2 Nephi,33,779,29399,120622,23,37,4.10
Jacob,7,203,9034,37786,29,44,4.18
Enos,1,27,1160,4730,27,42,4.08
Jarom,1,15,734,3149,15,48,4.29
Omni,1,30,1398,5906,30,46,4.22
Words of Mormon,1,18,827,3576,18,45,4.32
Mosiah,29,785,31159,130181,27,39,4.18
Alma,63,1973,84975,356151,31,43,4.19
Helaman,16,496,20414,85838,31,41,4.20


In [12]:
summarydf2

,chapters,verses,words,characters,V/C,W/V,Char/W
Total,239,6597,266569,1107416,403,616,62
Average,15,439,17771,73827,26,41,4


In [13]:
bookdf = pd.DataFrame(0,index=test.book.unique(),columns=['text'])

In [14]:
for x in test.book.unique():
    bookstring = ''
    temp = test[test.book == x].copy()
    for y in temp.verse:
        bookstring += y + ' '
    bookdf.loc[x,'text'] = bookstring

In [15]:
bookdf.text = bookdf.text.str.lower()

In [16]:
def find_word_in_book( wordtext ):
    global bookdf
    list_of_values = []
    starting_places = []
    for x in bookdf.index:
        templist = 0
        list_of_starts = []
        for word in wordtext:
            booktext = bookdf.loc[x,'text']
            starts = [m.start() for m in re.finditer(word,booktext)]
            templist += len(starts)
            list_of_starts += starts
        list_of_values.append(round(pd.Series(templist).sum()/float(summarydf.loc[x,'chapters']),2))
        starting_places.append(list_of_starts)
    return list_of_values,starting_places

In [17]:
search_string = ['repent','remission']
bookdf['repent'],bookdf['repentloc'] = find_word_in_book(search_string)

search_string = ['faith','trust','believe','belief','believing']
bookdf['faith'],bookdf['faithloc'] = find_word_in_book(search_string)

search_string = ['forgive','forgave']
bookdf['forgive'],bookdf['forgiveloc'] = find_word_in_book(search_string)

search_string = ['baptism','baptize']
bookdf['baptism'],bookdf['baptismloc'] = find_word_in_book(search_string)

search_string = ['holy ghost','spirit']
bookdf['spirit'],bookdf['spiritloc'] = find_word_in_book(search_string)

search_string = ['christ','lord','god','redeemer','savior','saviour','the son','messiah']
bookdf['God'],bookdf['Godloc'] = find_word_in_book(search_string)

In [18]:
bookdf[['repent','faith','baptism','spirit','God']]

,repent,faith,baptism,spirit,God
1 Nephi,0.36,1.64,0.36,2.41,20.55
2 Nephi,0.67,2.06,0.42,1.82,18.61
Jacob,0.71,1.71,0.00,1.71,20.71
Enos,1.00,11.00,0.00,1.00,23.00
Jarom,1.00,4.00,0.00,1.00,8.00
Omni,0.00,1.00,0.00,0.00,20.00
Words of Mormon,0.00,0.00,0.00,1.00,15.00
Mosiah,1.45,2.62,0.66,1.07,17.31
Alma,2.00,4.16,0.32,2.16,16.40
Helaman,4.25,3.13,0.38,1.00,14.81


In [27]:
bookdf.columns

Index([u'text', u'repent', u'repentloc', u'faith', u'faithloc', u'forgive',
       u'forgiveloc', u'baptism', u'baptismloc', u'spirit', u'spiritloc',
       u'God', u'Godloc'],
      dtype='object')

# Qualitative analysis of book of mormon topics

In [45]:
count

788

In [11]:
#stick these in a dataframe along with the reference that they come from
#how do I get the reference for these?
count = 0
for a in bookdf.index:
    for x in bookdf.loc[a,'faithloc']:
        print bookdf.loc[a,'text'][x-125:x+125]
        print
        count += 1